# Figma YouTube Channel Analysis and Insights Using YouTube API

## AIMS AND OBJECTIVES

1. Scrape publicly available video data using YouTube API.
2.  Clean and preprocess the dataset for analysis.
3. Categorize video titles into meaningful groups.
4. Perform exploratory data analysis to identify trends and patterns.
5. Use regression analysis to explore relationships between video metrics.
6. Present findings to support data-driven content strategy decisions.

Two tasks are perform in this project:
1. YouTube API data extraction/cleaning:

   
            - Create a new notebook
   
            - Create a Google Cloud Console account
   
            - Enable the YouTube API
   
            - Create new project and generate API key
   
            - Using the API key to connect to YouTube by enabling it
   
            - Importing necessary library
   
            - Extracting the data
   
            - data cleaning
   
2. Data modeling and analysis:

            - Exploratory Data Analysis (EDA)

            - Regression analysis : Predict views or likes based on published date, title length, etc.
   
            - Classification : If you categorize videos, predict category from title or metrics.

            - Clustering : Group similar videos by views, likes, comments, or text features.

            - Hypothesis testing: E.g., do videos published on certain days get more views?

# Task1 - YouTube API data extraction/cleaning 

In [6]:
# importing all necessary python library

import pandas as pd 
import numpy as ny 

In [7]:
api_key='Your_YouTube_API_KEY' #########PLEASE INPUT YOURS FOR THIS CODE TO RUN SUCCESSFULLY

In [8]:
pip install google-api-python-client pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install google-api-python-client pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
# importing build from google api library

from googleapiclient.discovery import build

In [11]:
# Set up API credentials
youtube = build('youtube', 'v3', developerKey=api_key)
channel_id = "UCQsVmhSa4X-G3lHlUtejzLA"

In [12]:
#Getting information about videos through channelid

def get_video_ids(youtube, channel_id):
    video_ids = []
    next_page_token = None

    while True:
        request = youtube.search().list(
            part="id",
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token,
            type="video",
            order="date"
        )
        response = request.execute()

        for item in response.get("items", []):
            if item["id"]["kind"] == "youtube#video":
                video_ids.append(item["id"]["videoId"])

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return video_ids


video_ids = get_video_ids(youtube, channel_id)
print(f"✅ Total videos found: {len(video_ids)}")

✅ Total videos found: 149


In [13]:
#  Getting Video Details

def get_video_details(youtube, video_ids):
    
    all_video_info = []
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,statistics",
            id=",".join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response.get('items', []):
            snippet = video['snippet']
            stats = video.get('statistics', {})

            video_info = {
                'Video ID': video['id'],
                'Title': snippet['title'],
                'Published Date': snippet['publishedAt'],
                'Views': int(stats.get('viewCount', 0)),
                'Likes': int(stats.get('likeCount', 0)),
                'Comments': int(stats.get('commentCount', 0))
            }
            all_video_info.append(video_info)

    return all_video_info


# Run and Save

video_ids = get_video_ids(youtube, channel_id)
print(f"✅ Total videos found: {len(video_ids)}")

video_data = get_video_details(youtube, video_ids)

df = pd.DataFrame(video_data)
df['Published Date'] = pd.to_datetime(df['Published Date'])

# Show all columns and rows by using set_option method

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Save to CSV

df.to_csv("zapier_youtube_data.csv", index=False)

print("✅ Data saved to zapier_youtube_data.csv")
print(df)

✅ Total videos found: 149
✅ Data saved to zapier_youtube_data.csv
        Video ID                                              Title  \
0    zuS8tPPwj9c  Cook up a noodle bowl illo in Figma Draw [Part...   
1    1y5bjxmXNfU  Cook up a noodle bowl illo in Figma Draw [Part...   
2    C-p5acGzwG0            Office Hours: Introducing Sites | Figma   
3    VJpGzNG2et4  Asking our execs to sign old photos of themsel...   
4    HCD8-JqCtH0     Figma for Edu: Smart animate with Figma Design   
5    Jp4A18QQz64               Office Hours: Introducing Figma Draw   
6    xwkuSspWcfI  How Astra Tech aligns design and dev across ME...   
7    1AOwk1N8cFs               Office Hours: Introducing Figma Make   
8    yw2T96J-dhI  How Figma helps On innovate at pace | Customer...   
9    zmNy95KSIIw               Office Hours: Introducing Figma Buzz   
10   qLyv9UO51bI           Release Notes 2025: July Edition | Figma   
11   rAlNWzpGiiE               New to variable width stroke | Figma   
12   Slm6gr

In [14]:
# Printing all the datas in the dataframe
df.head(147)

,Video ID,Title,Published Date,Views,Likes,Comments
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31
5,Jp4A18QQz64,Office Hours: Introducing Figma Draw,2025-08-07 13:00:39+00:00,829,26,2
6,xwkuSspWcfI,How Astra Tech aligns design and dev across ME...,2025-08-06 17:01:30+00:00,1920,37,0
7,1AOwk1N8cFs,Office Hours: Introducing Figma Make,2025-08-06 13:00:04+00:00,2453,72,3
8,yw2T96J-dhI,How Figma helps On innovate at pace | Customer...,2025-08-05 17:01:16+00:00,3154,59,0
9,zmNy95KSIIw,Office Hours: Introducing Figma Buzz,2025-08-05 13:41:52+00:00,1544,29,3


In [15]:
df

,Video ID,Title,Published Date,Views,Likes,Comments
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31
5,Jp4A18QQz64,Office Hours: Introducing Figma Draw,2025-08-07 13:00:39+00:00,829,26,2
6,xwkuSspWcfI,How Astra Tech aligns design and dev across ME...,2025-08-06 17:01:30+00:00,1920,37,0
7,1AOwk1N8cFs,Office Hours: Introducing Figma Make,2025-08-06 13:00:04+00:00,2453,72,3
8,yw2T96J-dhI,How Figma helps On innovate at pace | Customer...,2025-08-05 17:01:16+00:00,3154,59,0
9,zmNy95KSIIw,Office Hours: Introducing Figma Buzz,2025-08-05 13:41:52+00:00,1544,29,3


In [16]:
# Convert Published Date to datetime format
df["Published Date"] = pd.to_datetime(df["Published Date"], errors="coerce")

# Split into Date and Time columns
df["Date"] = df["Published Date"].dt.date
df["Time"] = df["Published Date"].dt.time


In [32]:
df.head()

,Video ID,Title,Published Date,Views,Likes,Comments,Date,Year,Month,Day,MonthName,Video Category
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0,2025-08-08,2025,8,8,Aug,Other
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1,2025-08-08,2025,8,8,Aug,Other
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2,2025-08-08,2025,8,8,Aug,Product Updates
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1,2025-08-07,2025,8,7,Aug,Community / Fun
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31,2025-08-07,2025,8,7,Aug,Education


In [18]:
# converting to date time
df["Date"] = pd.to_datetime(df["Date"])

# Extract Year, Month, Day
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day

# Extracting time column
df["Time"] = df["Date"].dt.time

In [31]:
df.head()

,Video ID,Title,Published Date,Views,Likes,Comments,Date,Year,Month,Day,MonthName,Video Category
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0,2025-08-08,2025,8,8,Aug,Other
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1,2025-08-08,2025,8,8,Aug,Other
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2,2025-08-08,2025,8,8,Aug,Product Updates
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1,2025-08-07,2025,8,7,Aug,Community / Fun
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31,2025-08-07,2025,8,7,Aug,Education


In [20]:
# dropping unuseful column e.g. time
df = df.drop(columns= 'Time')

In [30]:
df.head()

,Video ID,Title,Published Date,Views,Likes,Comments,Date,Year,Month,Day,MonthName,Video Category
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0,2025-08-08,2025,8,8,Aug,Other
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1,2025-08-08,2025,8,8,Aug,Other
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2,2025-08-08,2025,8,8,Aug,Product Updates
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1,2025-08-07,2025,8,7,Aug,Community / Fun
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31,2025-08-07,2025,8,7,Aug,Education


In [22]:
# Assign a month name to each day

def name_of_months(month):
    if month == 1:
        return "Jan"
    elif month == 2:
        return "Feb"
    elif month == 3: 
        return "Mar"
    elif month == 4:
        return "Apr"
    elif month == 5:
        return "May"
    elif month == 6:
        return "Jun"
    elif month == 7:
        return "Jul"
    elif month == 8:
        return "Aug"
    elif month == 9:
        return "Sep"
    elif month == 10:
        return "Oct"
    elif month == 11:
        return "Nov"
    elif month == 12:
        return "Dec"

# Applying the function to create a new column
df["MonthName"] = df["Month"].apply(name_of_months)

In [23]:
df.head(10)

,Video ID,Title,Published Date,Views,Likes,Comments,Date,Year,Month,Day,MonthName
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0,2025-08-08,2025,8,8,Aug
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1,2025-08-08,2025,8,8,Aug
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2,2025-08-08,2025,8,8,Aug
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1,2025-08-07,2025,8,7,Aug
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31,2025-08-07,2025,8,7,Aug
5,Jp4A18QQz64,Office Hours: Introducing Figma Draw,2025-08-07 13:00:39+00:00,829,26,2,2025-08-07,2025,8,7,Aug
6,xwkuSspWcfI,How Astra Tech aligns design and dev across ME...,2025-08-06 17:01:30+00:00,1920,37,0,2025-08-06,2025,8,6,Aug
7,1AOwk1N8cFs,Office Hours: Introducing Figma Make,2025-08-06 13:00:04+00:00,2453,72,3,2025-08-06,2025,8,6,Aug
8,yw2T96J-dhI,How Figma helps On innovate at pace | Customer...,2025-08-05 17:01:16+00:00,3154,59,0,2025-08-05,2025,8,5,Aug
9,zmNy95KSIIw,Office Hours: Introducing Figma Buzz,2025-08-05 13:41:52+00:00,1544,29,3,2025-08-05,2025,8,5,Aug


In [24]:
df.head(10)

,Video ID,Title,Published Date,Views,Likes,Comments,Date,Year,Month,Day,MonthName
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0,2025-08-08,2025,8,8,Aug
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1,2025-08-08,2025,8,8,Aug
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2,2025-08-08,2025,8,8,Aug
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1,2025-08-07,2025,8,7,Aug
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31,2025-08-07,2025,8,7,Aug
5,Jp4A18QQz64,Office Hours: Introducing Figma Draw,2025-08-07 13:00:39+00:00,829,26,2,2025-08-07,2025,8,7,Aug
6,xwkuSspWcfI,How Astra Tech aligns design and dev across ME...,2025-08-06 17:01:30+00:00,1920,37,0,2025-08-06,2025,8,6,Aug
7,1AOwk1N8cFs,Office Hours: Introducing Figma Make,2025-08-06 13:00:04+00:00,2453,72,3,2025-08-06,2025,8,6,Aug
8,yw2T96J-dhI,How Figma helps On innovate at pace | Customer...,2025-08-05 17:01:16+00:00,3154,59,0,2025-08-05,2025,8,5,Aug
9,zmNy95KSIIw,Office Hours: Introducing Figma Buzz,2025-08-05 13:41:52+00:00,1544,29,3,2025-08-05,2025,8,5,Aug


In [33]:
# Aim is to group all the titles in the title column by categories

def categorize_video(title):
    # this is to keep everything in lowercase for easy grouping
    title_lower = title.lower()
    if "edu" in title_lower or "education" in title_lower:
        return "Education"
    elif "figma make" in title_lower or "office hours" in title_lower:
        return "Product Updates"
    elif "execs" in title_lower or "photos" in title_lower:
        return "Community / Fun"
    elif "design" in title_lower or "dev" in title_lower:
        return "Design & Development"
    else:
        return "Other"

df["Video Category"] = df["Title"].apply(categorize_video)

print(df.head(10))

      Video ID                                              Title  \
0  zuS8tPPwj9c  Cook up a noodle bowl illo in Figma Draw [Part...   
1  1y5bjxmXNfU  Cook up a noodle bowl illo in Figma Draw [Part...   
2  C-p5acGzwG0            Office Hours: Introducing Sites | Figma   
3  VJpGzNG2et4  Asking our execs to sign old photos of themsel...   
4  HCD8-JqCtH0     Figma for Edu: Smart animate with Figma Design   
5  Jp4A18QQz64               Office Hours: Introducing Figma Draw   
6  xwkuSspWcfI  How Astra Tech aligns design and dev across ME...   
7  1AOwk1N8cFs               Office Hours: Introducing Figma Make   
8  yw2T96J-dhI  How Figma helps On innovate at pace | Customer...   
9  zmNy95KSIIw               Office Hours: Introducing Figma Buzz   

             Published Date  Views  Likes  Comments       Date  Year  Month  \
0 2025-08-08 13:32:42+00:00    250     10         0 2025-08-08  2025      8   
1 2025-08-08 13:32:50+00:00    351     26         1 2025-08-08  2025      8   
2 2

In [26]:
df.head(10)

,Video ID,Title,Published Date,Views,Likes,Comments,Date,Year,Month,Day,MonthName,Video Category
0,zuS8tPPwj9c,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:42+00:00,250,10,0,2025-08-08,2025,8,8,Aug,Other
1,1y5bjxmXNfU,Cook up a noodle bowl illo in Figma Draw [Part...,2025-08-08 13:32:50+00:00,351,26,1,2025-08-08,2025,8,8,Aug,Other
2,C-p5acGzwG0,Office Hours: Introducing Sites | Figma,2025-08-08 13:00:19+00:00,265,15,2,2025-08-08,2025,8,8,Aug,Product Updates
3,VJpGzNG2et4,Asking our execs to sign old photos of themsel...,2025-08-07 22:09:06+00:00,1237,38,1,2025-08-07,2025,8,7,Aug,Community / Fun
4,HCD8-JqCtH0,Figma for Edu: Smart animate with Figma Design,2025-08-07 16:13:36+00:00,2449,93,31,2025-08-07,2025,8,7,Aug,Education
5,Jp4A18QQz64,Office Hours: Introducing Figma Draw,2025-08-07 13:00:39+00:00,829,26,2,2025-08-07,2025,8,7,Aug,Product Updates
6,xwkuSspWcfI,How Astra Tech aligns design and dev across ME...,2025-08-06 17:01:30+00:00,1920,37,0,2025-08-06,2025,8,6,Aug,Design & Development
7,1AOwk1N8cFs,Office Hours: Introducing Figma Make,2025-08-06 13:00:04+00:00,2453,72,3,2025-08-06,2025,8,6,Aug,Product Updates
8,yw2T96J-dhI,How Figma helps On innovate at pace | Customer...,2025-08-05 17:01:16+00:00,3154,59,0,2025-08-05,2025,8,5,Aug,Other
9,zmNy95KSIIw,Office Hours: Introducing Figma Buzz,2025-08-05 13:41:52+00:00,1544,29,3,2025-08-05,2025,8,5,Aug,Product Updates


In [27]:
df.to_csv('youtube_data_cleaned.csv', index=False)

In [28]:
import os
print(os.getcwd())

/Users/einstein


In [29]:
# To shows the current working folder
!pwd

# To lists all files in that folder
!ls      

/Users/einstein
Desktop
Documents
Downloads
Figma_YouTube_content_performance_analysis.ipynb
Library
Movies
Music
Pictures
Public
anaconda_projects
figma_youtube_categorized.csv
youtube_data_cleaned.csv
youtube_videos.csv
zapier_youtube_data.csv
zapier_youtube_video_data.csv
